# 02: Prepare Data for Web

**Purpose:** Convert raw tile2net outputs to web-ready format

**Input:** `raw_output/manhattan_tile_X/YYYY/` (large shapefiles + 4x4 PNG grids)

**Output:** `web_ready/tiles/manhattan_tile_X/` (optimized PNGs + GeoJSON)

**What this does:**
1. Stitches 4x4 tile grids into single images (with transposition fix)
2. Optimizes images from ~4096px to 1024px (~30MB → ~2MB)
3. Converts shapefiles to GeoJSON (WGS84)
4. Generates metadata for frontend auto-discovery

## Setup & Configuration

In [30]:
# Install dependencies
!pip install -q geopandas pillow numpy

In [31]:
import geopandas as gpd
from pathlib import Path
from PIL import Image
import json
import numpy as np
from datetime import datetime
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# ============================================================
# CONFIGURATION
# ============================================================

BASE_PATH = Path('/content/drive/MyDrive/NYU/VizMLProject/manhattan_pipeline')
RAW_OUTPUT = BASE_PATH / 'raw_output'
WEB_READY = BASE_PATH / 'web_ready'
CONFIG_PATH = BASE_PATH / 'config/tiles_manhattan.json'

# Processing settings
TARGET_IMAGE_SIZE = 1024  # Resize images to 1024x1024 for web
GRID_SIZE = 4  # tile2net outputs 4x4 grids

# Which tiles/years to process
TILES_TO_PROCESS = [312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350] # Start with test tiles
YEARS_TO_PROCESS = [2024, 2022, 2020, 2018, 2016, 2014, 2012, 2010, 2008, 2006, 2004]     # Expand to [2004, 2007, 2010, ...] later

print("✅ Configuration loaded")
print(f"📂 Raw output: {RAW_OUTPUT}")
print(f"📂 Web ready: {WEB_READY}")
print(f"🎯 Processing {len(TILES_TO_PROCESS)} tiles for {len(YEARS_TO_PROCESS)} years")

✅ Configuration loaded
📂 Raw output: /content/drive/MyDrive/NYU/VizMLProject/manhattan_pipeline/raw_output
📂 Web ready: /content/drive/MyDrive/NYU/VizMLProject/manhattan_pipeline/web_ready
🎯 Processing 39 tiles for 11 years


## Load Tile Configuration

In [33]:
# Load tile grid configuration
with open(CONFIG_PATH, 'r') as f:
    tiles_config = json.load(f)

print(f"✅ Loaded config with {len(tiles_config['tiles'])} tiles")

# Create lookup by id
tiles_lookup = {t['id']: t for t in tiles_config['tiles']}

# Verify tiles exist
for id in TILES_TO_PROCESS:
    tile_name = f"manhattan_tile_{id}"
    if tile_name not in tiles_lookup:
        print(f"⚠️  Warning: {tile_name} not found in config")
    else:
        print(f"✓ {tile_name}: {tiles_lookup[tile_name]['name']}")

✅ Loaded config with 351 tiles
✓ manhattan_tile_312: Manhattan Tile 312 (R24C0)
✓ manhattan_tile_313: Manhattan Tile 313 (R24C1)
✓ manhattan_tile_314: Manhattan Tile 314 (R24C2)
✓ manhattan_tile_315: Manhattan Tile 315 (R24C3)
✓ manhattan_tile_316: Manhattan Tile 316 (R24C4)
✓ manhattan_tile_317: Manhattan Tile 317 (R24C5)
✓ manhattan_tile_318: Manhattan Tile 318 (R24C6)
✓ manhattan_tile_319: Manhattan Tile 319 (R24C7)
✓ manhattan_tile_320: Manhattan Tile 320 (R24C8)
✓ manhattan_tile_321: Manhattan Tile 321 (R24C9)
✓ manhattan_tile_322: Manhattan Tile 322 (R24C10)
✓ manhattan_tile_323: Manhattan Tile 323 (R24C11)
✓ manhattan_tile_324: Manhattan Tile 324 (R24C12)
✓ manhattan_tile_325: Manhattan Tile 325 (R25C0)
✓ manhattan_tile_326: Manhattan Tile 326 (R25C1)
✓ manhattan_tile_327: Manhattan Tile 327 (R25C2)
✓ manhattan_tile_328: Manhattan Tile 328 (R25C3)
✓ manhattan_tile_329: Manhattan Tile 329 (R25C4)
✓ manhattan_tile_330: Manhattan Tile 330 (R25C5)
✓ manhattan_tile_331: Manhattan Til

## Core Processing Functions

In [34]:
def stitch_tiles_with_transpose(tiles_folder, output_path, grid_size=4, target_size=None):
    """
    Stitch 4x4 grid of tiles into single image with transposition fix.

    CRITICAL: tile2net outputs tiles in a 4x4 grid with naming pattern:
    row_col_id.png where id = row * 4 + col

    BUT the tiles need to be transposed when stitching:
    canvas_row = source_col
    canvas_col = source_row

    Args:
        tiles_folder: Path to folder with 16 PNGs
        output_path: Where to save stitched image
        grid_size: Size of grid (4 = 4x4)
        target_size: If provided, resize final image to this size

    Returns:
        True if successful, False otherwise
    """
    try:
        # Get all PNG files and sort by numeric ID (not alphabetical!)
        tile_files = sorted(
            tiles_folder.glob("*.png"),
            key=lambda p: int(p.stem.split('_')[2])  # Extract ID from filename
        )

        if not tile_files:
            print("      ⚠️  No PNG files found")
            return False

        # Get dimensions from first tile
        first_tile = Image.open(tile_files[0])
        tile_size = first_tile.size[0]  # Assume square tiles
        first_tile.close()

        # Create canvas
        canvas_size = tile_size * grid_size
        canvas = Image.new('RGB', (canvas_size, canvas_size), color=(0, 0, 0))

        tiles_placed = 0

        # Process each tile
        for tile_path in tile_files:
            # Parse filename: row_col_id.png
            parts = tile_path.stem.split('_')
            if len(parts) < 3:
                continue

            tile_id = int(parts[2])

            # Calculate source position from ID
            source_row = tile_id // grid_size
            source_col = tile_id % grid_size

            # CRITICAL: Apply transposition
            canvas_row = source_col
            canvas_col = source_row

            # Verify position is valid
            if canvas_row >= grid_size or canvas_col >= grid_size:
                print(f"      ⚠️  Invalid position for tile {tile_id}: ({canvas_row}, {canvas_col})")
                continue

            # Paste tile onto canvas
            img = Image.open(tile_path)
            x_pos = canvas_col * tile_size
            y_pos = canvas_row * tile_size
            canvas.paste(img, (x_pos, y_pos))
            img.close()
            tiles_placed += 1

        # Resize if requested
        if target_size and target_size != canvas_size:
            canvas = canvas.resize(
                (target_size, target_size),
                Image.Resampling.LANCZOS
            )

        # Save
        output_path.parent.mkdir(parents=True, exist_ok=True)
        canvas.save(output_path, 'PNG', optimize=True)

        print(f"      ✅ Stitched {tiles_placed}/{len(tile_files)} tiles")
        if target_size:
            print(f"      📏 Resized to {target_size}x{target_size}")

        return True

    except Exception as e:
        print(f"      ❌ Stitching failed: {e}")
        return False

In [35]:
def convert_shapefile_to_geojson(shapefile_path, output_path):
    """
    Convert shapefile to GeoJSON in WGS84 projection.

    Args:
        shapefile_path: Path to .shp file
        output_path: Where to save .geojson

    Returns:
        Tuple of (success: bool, bounds: tuple or None)
    """
    try:
        # Read shapefile
        gdf = gpd.read_file(shapefile_path)

        # Convert to WGS84 if needed
        if gdf.crs and gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs(epsg=4326)

        # Save as GeoJSON
        output_path.parent.mkdir(parents=True, exist_ok=True)
        gdf.to_file(output_path, driver='GeoJSON')

        # Get bounds
        bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]

        print(f"      ✅ GeoJSON: {len(gdf)} features")

        return True, bounds

    except Exception as e:
        print(f"      ❌ GeoJSON conversion failed: {e}")
        return False, None

In [36]:
def find_shapefile(polygons_dir):
    """
    Find shapefile in polygons directory.
    tile2net outputs: polygons/polygon_crs_XXXX/*.shp
    """
    for subfolder in polygons_dir.iterdir():
        if subfolder.is_dir():
            shp_files = list(subfolder.glob("*.shp"))
            if shp_files:
                return shp_files[0]
    return None

def find_stitched_tiles(tiles_dir):
    """
    Find stitched tiles directory.
    tile2net outputs: tiles/stitched/stitched_tiles_XXXX/*.png
    """
    stitched_dir = tiles_dir / 'stitched'
    if not stitched_dir.exists():
        return None

    for subfolder in stitched_dir.iterdir():
        if subfolder.is_dir():
            png_files = list(subfolder.glob("*.png"))
            if png_files:
                return subfolder
    return None

## Process Tiles

In [37]:
def process_tile(tile_id, years):
    """
    Process a single tile for multiple years.

    Creates:
    - web_ready/tiles/manhattan_tile_X/imagery/YYYY.png
    - web_ready/tiles/manhattan_tile_X/networks/YYYY.geojson
    - web_ready/tiles/manhattan_tile_X/metadata.json
    """
    tile_name = f"manhattan_tile_{tile_id}"

    print(f"\n{'='*60}")
    print(f"🗺️  PROCESSING: {tile_name}")
    print(f"{'='*60}\n")

    # Get tile config
    tile_config = tiles_lookup.get(tile_name)
    if not tile_config:
        print(f"❌ Tile config not found")
        return None

    # Setup output directories
    tile_output = WEB_READY / 'tiles' / tile_name
    imagery_dir = tile_output / 'imagery'
    networks_dir = tile_output / 'networks'

    imagery_dir.mkdir(parents=True, exist_ok=True)
    networks_dir.mkdir(parents=True, exist_ok=True)

    # Track results
    successful_years = []
    bounds_list = []

    # Process each year
    for year in years:
        print(f"📅 {year}")

        # Find raw output directory
        year_dir = RAW_OUTPUT / tile_name / str(year) / f"{tile_name}_{year}"

        if not year_dir.exists():
            print(f"   ⚠️  Year directory not found: {year_dir}")
            continue

        year_success = {'imagery': False, 'geojson': False}

        # Process imagery
        tiles_folder = find_stitched_tiles(year_dir / 'tiles')
        if tiles_folder:
            print(f"   🖼️  Processing imagery...")
            output_img = imagery_dir / f"{year}.png"
            if stitch_tiles_with_transpose(tiles_folder, output_img,
                                          grid_size=GRID_SIZE,
                                          target_size=TARGET_IMAGE_SIZE):
                year_success['imagery'] = True
        else:
            print(f"   ⚠️  No stitched tiles found")

        # Process GeoJSON
        shapefile = find_shapefile(year_dir / 'polygons')
        if shapefile:
            print(f"   📊 Processing GeoJSON...")
            output_geojson = networks_dir / f"{year}.geojson"
            success, bounds = convert_shapefile_to_geojson(shapefile, output_geojson)
            if success:
                year_success['geojson'] = True
                bounds_list.append(bounds)
        else:
            print(f"   ⚠️  No shapefile found")

        # Record if any part succeeded
        if year_success['imagery'] or year_success['geojson']:
            successful_years.append(year)
            status = "✅" if (year_success['imagery'] and year_success['geojson']) else "⚠️"
            print(f"   {status} Year complete\n")
        else:
            print(f"   ❌ Year failed\n")

    # Generate metadata
    if successful_years:
        # Calculate bounds from GeoJSON data if available, otherwise use config
        if bounds_list:
            all_bounds = np.array(bounds_list)
            tile_bounds = {
                "west": float(all_bounds[:, 0].min()),
                "south": float(all_bounds[:, 1].min()),
                "east": float(all_bounds[:, 2].max()),
                "north": float(all_bounds[:, 3].max())
            }
        else:
            tile_bounds = tile_config['bounds']

        # Always use original config bounds for imagery positioning
        image_bounds = tile_config['bounds']

        # Calculate network bounds from actual GeoJSON data
        if bounds_list:
            all_bounds = np.array(bounds_list)
            network_bounds = {
                "west": float(all_bounds[:, 0].min()),
                "south": float(all_bounds[:, 1].min()),
                "east": float(all_bounds[:, 2].max()),
                "north": float(all_bounds[:, 3].max())
            }
        else:
            network_bounds = None

        metadata = {
            "tile_id": tile_name,
            "name": tile_config['name'],
            "bounds": image_bounds,  # ← KEEP original config bounds for imagery!
            "network_bounds": network_bounds,  # ← Add separate network bounds for reference
            "years": sorted(successful_years, reverse=True),
            "processed_date": datetime.now().isoformat()
        }

        with open(tile_output / 'metadata.json', 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"✅ {tile_config['name']}: {len(successful_years)} years processed")

        return {
            'tile_id': tile_name,
            'name': tile_config['name'],
            'years': successful_years
        }
    else:
        print(f"❌ No years successfully processed")
        return None

In [ ]:
# Process all tiles
print("\n" + "="*60)
print("🚀 STARTING PROCESSING")
print("="*60 + "\n")

results = []

for i, tile_id in enumerate(TILES_TO_PROCESS, 1):
    print(f"\n[{i}/{len(TILES_TO_PROCESS)}] Processing tile {tile_id}...")
    result = process_tile(tile_id, YEARS_TO_PROCESS)
    if result:
        results.append(result)

## Generate Tiles Index

In [39]:
def generate_tiles_index():
    """
    Generate tiles_index.json for frontend auto-discovery.

    CRITICAL: Uses 'bounds' (original config) NOT 'network_bounds' for imagery positioning.

    Creates: web_ready/tiles_index.json

    Format:
    [
      {
        "tile_id": "manhattan_tile_0",
        "name": "Financial District",
        "bounds": {"north": ..., "south": ..., "west": ..., "east": ...}
      }
    ]
    """
    tiles_dir = WEB_READY / 'tiles'
    tiles_index = []

    # Scan for all tiles with metadata
    for tile_dir in sorted(tiles_dir.iterdir()):
        if tile_dir.is_dir():
            meta_path = tile_dir / 'metadata.json'
            if meta_path.exists():
                with open(meta_path, 'r') as f:
                    metadata = json.load(f)

                    # CRITICAL: Always use 'bounds' field (original config bounds)
                    # DO NOT use 'network_bounds' - that's only for reference
                    tile_entry = {
                        "tile_id": metadata['tile_id'],
                        "name": metadata['name'],
                        "bounds": metadata['bounds']  # Original config bounds for imagery
                    }

                    tiles_index.append(tile_entry)

                    # Validation: Check if bounds look reasonable
                    b = metadata['bounds']
                    lat_range = b['north'] - b['south']
                    lon_range = b['east'] - b['west']

                    # Warn if bounds seem too small (might be network_bounds by mistake)
                    if lat_range < 0.005 or lon_range < 0.005:
                        print(f"⚠️  {metadata['tile_id']}: bounds seem small")
                        print(f"    Lat range: {lat_range:.6f}°, Lon range: {lon_range:.6f}°")
                        if 'network_bounds' in metadata and metadata['network_bounds']:
                            nb = metadata['network_bounds']
                            print(f"    (network_bounds would be: {nb['north']-nb['south']:.6f}° x {nb['east']-nb['west']:.6f}°)")

    # Save index
    index_path = WEB_READY / 'tiles_index.json'
    with open(index_path, 'w') as f:
        json.dump(tiles_index, f, indent=2)

    print(f"\n✅ Generated tiles_index.json with {len(tiles_index)} tiles")
    print(f"📁 Location: {index_path}")

    # Print summary for verification
    print(f"\n🔍 Tiles index summary:")
    for tile in tiles_index:
        b = tile['bounds']
        lat_range = b['north'] - b['south']
        lon_range = b['east'] - b['west']
        print(f"  {tile['tile_id']}: {lat_range:.6f}° x {lon_range:.6f}°")

    return tiles_index

# Generate index
print("\n" + "="*60)
print("📋 GENERATING TILES INDEX")
print("="*60 + "\n")

tiles_index = generate_tiles_index()


📋 GENERATING TILES INDEX


✅ Generated tiles_index.json with 350 tiles
📁 Location: /content/drive/MyDrive/NYU/VizMLProject/manhattan_pipeline/web_ready/tiles_index.json

🔍 Tiles index summary:
  manhattan_tile_0: 0.006757° x 0.008824°
  manhattan_tile_1: 0.006757° x 0.008824°
  manhattan_tile_10: 0.006757° x 0.008824°
  manhattan_tile_100: 0.006757° x 0.008824°
  manhattan_tile_101: 0.006757° x 0.008824°
  manhattan_tile_102: 0.006757° x 0.008824°
  manhattan_tile_103: 0.006757° x 0.007171°
  manhattan_tile_104: 0.006757° x 0.008824°
  manhattan_tile_105: 0.006757° x 0.008824°
  manhattan_tile_106: 0.006757° x 0.008824°
  manhattan_tile_107: 0.006757° x 0.008824°
  manhattan_tile_108: 0.006757° x 0.008824°
  manhattan_tile_109: 0.006757° x 0.008824°
  manhattan_tile_11: 0.006757° x 0.008824°
  manhattan_tile_110: 0.006757° x 0.008824°
  manhattan_tile_111: 0.006757° x 0.008824°
  manhattan_tile_112: 0.006757° x 0.008824°
  manhattan_tile_113: 0.006757° x 0.008824°
  manhattan_tile_114